# CABEÇALHO

**Descrição:** este algoritmo foi elaborado para auxílio no projeto de pesquisa dos alunos Luiz Fonseca e Gabriela Ricardino, para o ano de 2023, do curso de Eng. de Energias Renováveis - UFPB, sob orientação dos professores Monica Carvalho e Raphael Abrahão.

**Objetivo:** auxílio no tratamento de dados meteorológicos obtidos através dos sites: 
  - [BDMEP](https://bdmep.inmet.gov.br/)
  - [NASA](https://power.larc.nasa.gov/data-access-viewer/)


**Desenvolvedor:** Luiz Fonseca [luiz.fonseca@cear.ufpb.br]

**Ano de desenvolvimento:** 2023

# IMPORTANTE

**Instruções, em vídeo, para uso do algoritmo:** https://youtu.be/M9Vh9MmBEmA
___

**Instruções para uso do algoritmo:**
1. Os arquivos **DEVEM** ser com extensão .csv;
2. Importe o arquivo no ícone de 'pasta' do lado esquerdo do Google Colab (quarto ícone de cima para baixo);
3. Para dados do INMET, são aceitos: dados diários ou dados mensais. Para dados da NASA: dados diários;
4. O tipo de pontuação para separação de decimais, para dados do INMET, **DEVE** ser PONTO;
5. Substitua, no algoritmo abaixo, os valores das variáveis entre as aspas simples:
  - variável *fonte*: os valores possíveis são **nasa** ou **inmet**, dentro das aspas simples;
  - variável *tipo_de_dado*: os valores possíveis são **diario** ou **mensal**, dentro das aspas simples;
  - variável *local_arquivo*: **path** do arquivo o qual você deseja trabalhar. Na pasta à esquerda, onde foi feito o upload do arquivo, basta clicar com o botão direito sobre ele e cliclar em 'Copiar caminho'. Substitua o que foi copiado entre as aspas simples.
6. Execute o algoritmo clicando no símbolo Screenshot_2.png no canto superior esquerdo do bloco;
7. O arquivo processado estará disponível na mesma pasta onde foi feito o upload do arquivo original, à esquerda. O nome do arquivo processado aparecerá como: 'Arquivo_Processado_[nome-do-arquivo-original].csv'.


> **OBS:** O algoritmo faz avaliação simultânea de todas as variáveis contidas no mesmo arquivo. Caso você queira avaliar as variáveis individualmente, separe-as em arquivos individuais, faça upload e execute o algoritmo para cada arquivo, individualmente.



# ALGORITMO

In [5]:
fonte =             'nasa'  # possíveis opções: 'nasa' ou 'inmet'
tipo_de_dado =      'diario'   # possíveis opções: 'diario' ou 'mensal'
local_arquivo =     '/content/joao_pessoa_irradiacao_diario.csv'

'''

      Por favor, não altere as linhas de código abaixo desta linha!

'''

import pandas as pd
import numpy as np

def dados_diarios_inmet():
  df = pd.read_csv(local_arquivo, sep = ';', skiprows = 10, decimal = '.') 
  df['Data Medicao'] = pd.to_datetime(df['Data Medicao'], dayfirst = True)
  num_colunas = df.shape[1]
  df_temp = pd.DataFrame({df.columns[-1]:['STOP']})
  df = df.append(df_temp, ignore_index=True)
  i = 0
  lista_indices = []
  while(str(df.iat[i,-1]) != 'STOP'):
    for k in range(1,num_colunas-1):
      if (str(df.iat[i,k]) == 'nan'):
        lista_indices.append(i)
        break;
      else:
        continue;
    i += 1
    if (str(df.iat[i,-1]) == 'STOP'):
      break;
  df.drop(df.index[lista_indices], inplace=True) 
  df.reset_index()
  df['Meses'] = df['Data Medicao'].dt.month
  df['Anos'] = df['Data Medicao'].dt.year
  lista_indices_2 = []
  df_qtd_dias_por_mes_e_ano = df.groupby(['Anos','Meses']).count()
  for i in range (0, len(df_qtd_dias_por_mes_e_ano)):
    ano = str(int(df_qtd_dias_por_mes_e_ano.index[i][0]))
    mes = str(int(df_qtd_dias_por_mes_e_ano.index[i][1]))
    qntd_correta_dias = pd.Period(ano + '-' + mes).days_in_month
    if df_qtd_dias_por_mes_e_ano.iloc[i,0] != qntd_correta_dias:
      lista_indices_2.append(str(float(mes)) + '-' + str(float(ano)))
  lista_mes_e_ano = []
  for i in range (len(df)):
    lista_mes_e_ano.append(str(df.iloc[i,-2]) + '-' + str(df.iloc[i,-1]))
  df['Meses+Anos'] = lista_mes_e_ano
  lista_indices_3 = []
  for k in range(len(df)):
    if (str(df.iat[k,-4]) == 'STOP'):
      break;
    if (df.iat[k,-1] in lista_indices_2):
      lista_indices_3.append(k)
    else:
      continue;
  df.drop(df.index[lista_indices_3], inplace=True)
  df_media_por_mes_e_ano = round(df.groupby(['Meses','Anos'],sort=False).mean(), 2)
  df_novo = pd.DataFrame(index = np.arange(0), columns = np.arange(12))
  df_cabecalho = pd.read_csv(local_arquivo, nrows = 10, usecols = [0], sep = ';', decimal = '.')
  df_novo = pd.concat([df_cabecalho,df_novo], sort = False)
  nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
  x = 1
  h = 0
  condicao_inicial = True
  while (condicao_inicial):
    nova_linha_cabecalho = [df.columns.values[x]]   
    if ('Unnamed: ' + str(x)) in nova_linha_cabecalho: 
      break;
    else:
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False)
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False) 
      f = 0 
      b = 0 
      u = 0 
      p = 0
      cond = True
      planilha_final = []
      while(cond):
        ano_temp = df_media_por_mes_e_ano.index[f][1]
        mes_para_linha = [] 
        mes_para_planilha = []
        while(df_media_por_mes_e_ano.index[b][1] == ano_temp): 
          mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h])
          b += 1
          if (len(df_media_por_mes_e_ano) == b):
            break;      
        for i in range (0,13):
          if p >= len(df_media_por_mes_e_ano): 
            mes_para_planilha.append('-')
            p += 1
            continue;
          if i == 0:
            mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
            continue;
          if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
            if (len(mes_para_linha) == u):
              continue;
            else:
              mes_para_planilha.append(mes_para_linha[u])
              u += 1
              p += 1
          else:
            mes_para_planilha.append('-')
            continue;
        u = 0    
        planilha_final.append(mes_para_planilha) 
        f = b 
        if (len(df_media_por_mes_e_ano) == f):
          cond = False
          break;
      df_para_adicionar = pd.DataFrame(planilha_final)
      df_novo = df_novo.append(df_para_adicionar, ignore_index=True)
      pula_linha = ['*','*','*']
      df_pula_linha = pd.DataFrame(pula_linha)
      df_novo = df_novo.append(df_pula_linha, ignore_index=True)
      h += 1 
      x += 1
      if ('Unnamed: ' + str(x)) in nova_linha_cabecalho:
        condicao_inicial = False
        break;
  df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep=';', decimal = '.', index = False)

def dados_mensais_inmet():
  df = pd.read_csv(local_arquivo, sep=';', skiprows=10, decimal='.')
  df['Data Medicao'] = pd.to_datetime(df['Data Medicao'],dayfirst=True)
  num_colunas = df.shape[1]
  df_temp = pd.DataFrame({df.columns[-1]:['STOP']})
  df = df.append(df_temp, ignore_index=True)
  i = 0
  lista_indices = []
  while(str(df.iat[i,-1]) != 'STOP'):
    for k in range(1,num_colunas-1):
      if (str(df.iat[i,k]) == 'nan'):
        lista_indices.append(i)
        break;
      else:
        continue;
    i += 1
    if (str(df.iat[i,-1]) == 'STOP'):
      break;
  df.drop(df.index[lista_indices], inplace=True) 
  df.reset_index()
  df['Meses'] = df['Data Medicao'].dt.month
  df['Anos'] = df['Data Medicao'].dt.year
  lista_mes_e_ano = []
  for i in range (len(df)):
    lista_mes_e_ano.append(str(df.iloc[i,-2]) + '-' + str(df.iloc[i,-1]))
  df['Meses+Anos'] = lista_mes_e_ano
  df_media_por_mes_e_ano = round(df.groupby(['Meses','Anos'],sort=False).mean(), 2)
  df_novo = pd.DataFrame (index=np.arange(0), columns=np.arange(12))
  df_cabecalho = pd.read_csv(local_arquivo, nrows=10, usecols=[0], sep=';', decimal='.')
  df_novo = pd.concat([df_cabecalho,df_novo], sort=False)
  nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
  x = 1
  h = 0
  condicao_inicial = True
  while (condicao_inicial):
    nova_linha_cabecalho = [df.columns.values[x]]  
    if ('Unnamed: ' + str(x)) in nova_linha_cabecalho:
      break;
    else:
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False)
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False)
      f = 0
      b = 0
      u = 0
      p = 0 
      cond = True
      planilha_final = []
      while(cond):
        ano_temp = df_media_por_mes_e_ano.index[f][1] 
        mes_para_linha = []
        mes_para_planilha = []
        while(df_media_por_mes_e_ano.index[b][1] == ano_temp):
          mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h])
          b += 1
          if (len(df_media_por_mes_e_ano) == b):
            break;    
        for i in range (0,13):
          if p >= len(df_media_por_mes_e_ano): 
            mes_para_planilha.append('-')
            p += 1
            continue;
          if i == 0:
            mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
            continue;
          if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
            if (len(mes_para_linha) == u):
              continue;
            else:
              mes_para_planilha.append(mes_para_linha[u])
              u += 1
              p += 1
          else:
            mes_para_planilha.append('-')
            continue;
        u = 0
        planilha_final.append(mes_para_planilha)
        f = b 
        if (len(df_media_por_mes_e_ano) == f):
          cond = False
          break;
      df_para_adicionar = pd.DataFrame(planilha_final)
      df_novo = df_novo.append(df_para_adicionar, ignore_index=True)
      pula_linha = ['*','*','*']
      df_pula_linha = pd.DataFrame(pula_linha)
      df_novo = df_novo.append(df_pula_linha, ignore_index=True)
      h += 1 
      x += 1
      if ('Unnamed: ' + str(x)) in nova_linha_cabecalho:
        condicao_inicial = False
        break;
  df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep=';', decimal = '.', index = False)

def dados_diarios_nasa():
  df = pd.read_csv(local_arquivo,sep=',',skiprows=9, decimal='.')
  if df.shape[1] != 4:
    print('Por favor, insira um arquivo com apenas 1 (uma) variável retirado do site da NASA.')
  else:
    df['STOP'] = 'STOP'
    num_colunas = df.shape[1]
    i = 0
    f = 0
    lista_indices = []
    while(f < len(df)):
      for k in range(3,num_colunas):
        if (df.iat[i,k] == -999.0):
          lista_indices.append(i)
          break;
        else:
          continue;
      i += 1
      f += 1
      if (f == len(df)):
        break;
    df.drop(df.index[lista_indices], inplace=True)
    df.reset_index()
    lista_indices_2 = []
    df_qtd_dias_por_mes_e_ano = df.groupby(['YEAR','MO']).count()
    for i in range (0, len(df_qtd_dias_por_mes_e_ano)):
      ano = str(int(df_qtd_dias_por_mes_e_ano.index[i][0]))
      mes = str(int(df_qtd_dias_por_mes_e_ano.index[i][1]))
      qntd_correta_dias = pd.Period(ano + '-' + mes).days_in_month
      if df_qtd_dias_por_mes_e_ano.iloc[i,0] != qntd_correta_dias:
        lista_indices_2.append(str(int(ano)) + '-' + str(int(mes)))
    lista_mes_e_ano = []
    for i in range (len(df)):
      lista_mes_e_ano.append(str(int(df.iloc[i,0])) + '-' + str(int(df.iloc[i,1])))
    df['Meses+Anos'] = lista_mes_e_ano
    lista_indices_3 = [] 
    for k in range(len(df)):
      if (df.iat[k,-1] in lista_indices_2):
        lista_indices_3.append(k)
      else:
        continue;
    df.drop(df.index[lista_indices_3], inplace=True)
    df_media_por_mes_e_ano = round(df.groupby(['MO','YEAR'],sort=False).mean(), 2)
    df_novo = pd.DataFrame (index=np.arange(0), columns=np.arange(12))
    df_cabecalho = pd.read_csv(local_arquivo, nrows=9, usecols=[0], sep=',', decimal='.')
    df_novo = pd.concat([df_cabecalho,df_novo], sort=False)
    nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
    x = 3
    h = 1
    condicao_inicial = True
    while (condicao_inicial):
      nova_linha_cabecalho = [df.columns.values[x]]
      if ('STOP' in nova_linha_cabecalho):
        break;
      else:
        df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False)
        df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False)
        f = 0 
        b = 0
        u = 0
        p = 0 
        cond = True
        planilha_final = [] 
        while(cond):
          ano_temp = df_media_por_mes_e_ano.index[f][1]
          mes_para_linha = []
          mes_para_planilha = []
          while(df_media_por_mes_e_ano.index[b][1] == ano_temp):
            mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h])
            b += 1
            if (len(df_media_por_mes_e_ano) == b):
              break; 
          for i in range (0,13):
            if p >= len(df_media_por_mes_e_ano): 
              mes_para_planilha.append('-')
              p += 1
              continue;
            if i == 0:
              mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
              continue;
            if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
              if (len(mes_para_linha) == u):
                continue;
              else:
                mes_para_planilha.append(mes_para_linha[u])
                u += 1
                p += 1
            else:
              mes_para_planilha.append('-')
              continue;
          u = 0
          planilha_final.append(mes_para_planilha)
          f = b        
          if (len(df_media_por_mes_e_ano) == f):
            cond = False
            break;
        df_para_adicionar = pd.DataFrame(planilha_final)
        df_novo = df_novo.append(df_para_adicionar, ignore_index=True)
        ultima_linha = ['*','*','*']
        df_ultima_linha = pd.DataFrame(ultima_linha)
        df_novo = df_novo.append(df_ultima_linha, ignore_index=True)
        h += 1
        x += 1
        if ('STOP' in nova_linha_cabecalho): 
          condicao_inicial = False
          break;
    df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep = ';', decimal = '.', index = False)

fonte = fonte.lower().strip()
tipo_de_dado = tipo_de_dado.lower().strip().replace('á','a')

if fonte == 'nasa' and tipo_de_dado == 'diario':
    dados_diarios_nasa()
elif fonte == 'inmet' and tipo_de_dado == 'diario':
  dados_diarios_inmet()
elif fonte == 'inmet' and tipo_de_dado == 'mensal':
  dados_mensais_inmet()
else:
  print('ERRO! Verifique se você inseriu as informações de maneira correta nas variáveis: "fonte", "tipo_de_dado" e "local_arquivo".')